In [1]:
pip install -q catboost


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -q scikit-learn catboost pandas numpy


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [5]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(train_df.shape, test_df.shape)
train_df.head()


(700000, 26) (300000, 25)


,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,...,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,...,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,...,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,...,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0,1.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,...,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0,1.0


In [6]:
TARGET = "diagnosed_diabetes"

X = train_df.drop(columns=[TARGET])
y = train_df[TARGET]

X_test_final = test_df.copy()


In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [10]:
cat_features = X.select_dtypes(include=["object", "category"]).columns.tolist()
print(cat_features)


['gender', 'ethnicity', 'education_level', 'income_level', 'smoking_status', 'employment_status']


In [17]:
model = CatBoostClassifier(
    iterations=5000,
    learning_rate=0.02,
    depth=8,
    loss_function="Logloss",
    eval_metric="AUC",
    task_type="GPU",
    devices="0",
    cat_features=cat_features,
    random_seed=42,
    verbose=200
)


In [18]:
model.fit(
    X_train,
    y_train,
    eval_set=(X_valid, y_valid),
    use_best_model=True
)


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6856524	best: 0.6856524 (0)	total: 39.5ms	remaining: 3m 17s
200:	test: 0.7070169	best: 0.7070169 (200)	total: 7.42s	remaining: 2m 57s
400:	test: 0.7122654	best: 0.7122654 (400)	total: 14.9s	remaining: 2m 51s
600:	test: 0.7157011	best: 0.7157011 (600)	total: 22.4s	remaining: 2m 44s
800:	test: 0.7187092	best: 0.7187092 (800)	total: 29.9s	remaining: 2m 36s
1000:	test: 0.7202885	best: 0.7202885 (1000)	total: 37.3s	remaining: 2m 29s
1200:	test: 0.7214149	best: 0.7214149 (1200)	total: 44.9s	remaining: 2m 21s
1400:	test: 0.7220890	best: 0.7220890 (1400)	total: 52.3s	remaining: 2m 14s
1600:	test: 0.7226088	best: 0.7226088 (1600)	total: 59.9s	remaining: 2m 7s
1800:	test: 0.7230442	best: 0.7230442 (1800)	total: 1m 7s	remaining: 1m 59s
2000:	test: 0.7233163	best: 0.7233192 (1990)	total: 1m 15s	remaining: 1m 52s
2200:	test: 0.7235747	best: 0.7235747 (2200)	total: 1m 22s	remaining: 1m 45s
2400:	test: 0.7237825	best: 0.7237869 (2385)	total: 1m 30s	remaining: 1m 37s
2600:	test: 0.7239276	b

In [19]:
valid_preds = model.predict_proba(X_valid)[:, 1]
auc = roc_auc_score(y_valid, valid_preds)

print(f"Validation ROC-AUC: {auc:.5f}")


Validation ROC-AUC: 0.72463


In [20]:
model.fit(
    X,
    y,
    verbose=200
)


Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 54.4ms	remaining: 4m 31s
200:	total: 8.46s	remaining: 3m 22s
400:	total: 16.7s	remaining: 3m 11s
600:	total: 24.9s	remaining: 3m 2s
800:	total: 33.2s	remaining: 2m 53s
1000:	total: 41.5s	remaining: 2m 45s
1200:	total: 49.8s	remaining: 2m 37s
1400:	total: 58.1s	remaining: 2m 29s
1600:	total: 1m 6s	remaining: 2m 21s
1800:	total: 1m 14s	remaining: 2m 12s
2000:	total: 1m 23s	remaining: 2m 4s
2200:	total: 1m 31s	remaining: 1m 56s
2400:	total: 1m 39s	remaining: 1m 48s
2600:	total: 1m 48s	remaining: 1m 39s
2800:	total: 1m 56s	remaining: 1m 31s
3000:	total: 2m 4s	remaining: 1m 23s
3200:	total: 2m 13s	remaining: 1m 14s
3400:	total: 2m 21s	remaining: 1m 6s
3600:	total: 2m 29s	remaining: 58.1s
3800:	total: 2m 37s	remaining: 49.8s
4000:	total: 2m 46s	remaining: 41.5s
4200:	total: 2m 54s	remaining: 33.2s
4400:	total: 3m 2s	remaining: 24.9s
4600:	total: 3m 11s	remaining: 16.6s
4800:	total: 3m 19s	remaining: 8.28s
4999:	total: 3m 28s	remaining: 0us


In [21]:
test_preds = model.predict_proba(X_test_final)[:, 1]


In [22]:
submission = pd.DataFrame({
    "id": test_df["id"],
    "diagnosed_diabetes": test_preds
})

submission.to_csv("submission.csv", index=False)
submission.head()


,id,diagnosed_diabetes
0,700000,0.510306
1,700001,0.673657
2,700002,0.774044
3,700003,0.417566
4,700004,0.910698
